In [14]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import prepState
from numpy import linalg as LA
from numpy import log as ln
from timeEvo import *
from qutip import *

# for using tex formatting and font in plots
#"""
plt.rcParams.update({"text.usetex": True,}) 
mpl.rcParams['text.latex.preamble'] = [r'\usepackage[utf8]{inputenc}\usepackage[T1]{fontenc}\usepackage{lmodern}\inputencoding{utf8}\usepackage{amsmath}\usepackage{amssymb}\usepackage{dsfont}\usepackage{mathtools}\usepackage{physics}']
mpl.rc('font', **{'family':'sans-serif', 'size':11})
cmap = mpl.cm.get_cmap('viridis')#['viridis', 'plasma', 'inferno', 'magma', 'cividis']

In [15]:
N=4
H_perf = generateStates(N,"perf")
hbar = ln(2)**2
n = 3200
dt = 1e-3
t = np.linspace(0, n*dt, n)

In [16]:
def thermal_state(beta):
    rho = (-beta * sigmaz()).expm()
    Z = rho.tr()
    rho = rho/Z
    return rho


def writetxt(filename, quantity):
    quantity = np.real(quantity)
    with open(str(filename) + ".txt", "w") as f:
        for element in quantity:
            f.write(str(element) + '\n')

In [19]:
## \dot{I}^2 max value as a function of alpha and position of max fidelity
def three_maxima(beta_one, beta_two, filename):
    alpha = 1.j / (4*np.cosh(beta_one)*np.cosh(beta_two))
    chi = alpha * tensor(Qobj([[0,1],[0,0]]), Qobj([[0,0],[1,0]])) - alpha * tensor(Qobj([[0,0],[1,0]]), Qobj([[0,1],[0,0]]))
    first_second_state = tensor(thermal_state(beta_one), thermal_state(beta_two))

    var_alpha_evo = []
    ratio = np.append(np.logspace(-3,-1.1,15, endpoint=True), np.linspace(.1, 1, 30, endpoint=True))
    ratio = np.insert(ratio, 0,0)

    ground_state = Qobj([[0,0],[0,1]])

    for i in ratio:
        corr_state = first_second_state + chi*i
        var_alpha_system = tensor(corr_state, ground_state, ground_state, ground_state)
        time_evo = qutip.mesolve(H_perf, var_alpha_system, t, [], []).states
        var_alpha_evo.append(time_evo)

    ind = 0
    print("time evo done")
    print()

    filename = "verkackt/maxima_with_diff_beta"+filename

    print(filename)
    print('dot{I}^2_max \t t^star \t V_max \t alpha/alpha_max \t beta1 \t beta2')
    with open("txtfiles/i_dot_sq_max/"+filename+".txt", 'a') as f:
        f.write('dot{I}^2_max \t t^star \t V_max \t alpha/alpha_max \t beta1 \t beta2\n')
        for alpha_evo in var_alpha_evo:
            bound = list(np.real([-1.j *(np.pi/(3*hbar)) * (commutator(H_perf, alpha_evo[dt]).ptrace(N) * (sigmaz())).tr() for dt in range(1,len(t))]))

            break_values = bound[1000:3140]#np.isclose(bound[1000:], 0, atol=0.005)
            break_values = break_values[break_values <= 0]
            t_bound = bound.index(break_values[0])
            f.write(str(t_bound) +'\t'+str(ratio[ind])+'\t'+str(beta_one)+'\t'+str(beta_two)+'\n')
            print(str(t_bound)+'\t'+str(ratio[ind])+'\t'+str(beta_one)+'\t'+str(beta_two))
            ind += 1


In [20]:
betas = [1/300, .25, .5, .75, 1.21, 10]
ij = [(0,0), (0,1), (0,2), (0,3), (0,4), (0,5)]
for i, j in ij:
    three_maxima(beta_one=betas[i], beta_two=betas[j], filename=str(i)+str(j))


In [34]:
def which_beta_exact(start, inc):
    beta_one = 1/300
    var_alpha_evo = []
    for i in range(0,11):
        beta_two = start + inc*i
        
        first_second_state = tensor(thermal_state(beta_one), thermal_state(beta_two))

        ground_state = Qobj([[0,0],[0,1]])

        var_alpha_system = tensor(first_second_state, ground_state, ground_state, ground_state)
        time_evo = qutip.mesolve(H_perf, var_alpha_system, t, [], []).states
        var_alpha_evo.append(time_evo)
    
    i_dot_sq_max = []
    
    for alpha_evo in var_alpha_evo:
        vn_alpha = [entropy_vn(alpha_evo[dt].ptrace(N), base=np.e) for dt in range(len(t))]

        i_dot_sq_alpha = np.square(np.diff(vn_alpha)/dt)
        i_dot_sq_max.append(max(i_dot_sq_alpha[:3142]))
    
    i_dot_diff = np.diff(i_dot_sq_max)
    ind = np.where(i_dot_diff == np.min(i_dot_diff))
    print(ind)

    if int(inc*1e4) == 0 and np.min(i_dot_diff)<0:
        return start + inc*i
    elif int(inc*1e4) >0 and np.min(i_dot_diff)<0:
        which_beta_exact(start = start + inc*(ind-1), inc = .2*inc)
    elif np.min(i_dot_diff)>0 and int(inc*1e4)>0:
        which_beta_exact(start = start - inc*np.float64(ind), inc = .2*inc)


print(which_beta_exact(.85, .001))

(array([0]),)


ValueError: dimension mismatch

In [ ]:
def which_beta(filename, corr):
    beta_one = 1/300
    var_alpha_evo = []
    if corr=='uncorr':
        for i in range(0,50):
            beta_two = .1 + 0.1*i
            
            first_second_state = tensor(thermal_state(beta_one), thermal_state(beta_two))

            ground_state = Qobj([[0,0],[0,1]])

            var_alpha_system = tensor(first_second_state, ground_state, ground_state, ground_state)
            time_evo = qutip.mesolve(H_perf, var_alpha_system, t, [], []).states
            var_alpha_evo.append(time_evo)
    elif corr=='max':
        for i in range(0,11):
            beta_two = .83 + 0.01*i
            alpha = 1.j / (4*np.cosh(beta_one)*np.cosh(beta_two))
            chi = alpha * tensor(Qobj([[0,1],[0,0]]), Qobj([[0,0],[1,0]])) - alpha * tensor(Qobj([[0,0],[1,0]]), Qobj([[0,1],[0,0]]))
            first_second_state = tensor(thermal_state(beta_one), thermal_state(beta_two))

            ground_state = Qobj([[0,0],[0,1]])


            corr_state = first_second_state + chi
            var_alpha_system = tensor(corr_state, ground_state, ground_state, ground_state)
            time_evo = qutip.mesolve(H_perf, var_alpha_system, t, [], []).states
            var_alpha_evo.append(time_evo)
    elif type(corr) != str and corr < 1 and corr > 0:
        for i in range(0,11):
            beta_two = .82 + 0.001*i

            alpha = 1.j / (4*np.cosh(beta_one)*np.cosh(beta_two))
            chi = alpha * tensor(Qobj([[0,1],[0,0]]), Qobj([[0,0],[1,0]])) - alpha * tensor(Qobj([[0,0],[1,0]]), Qobj([[0,1],[0,0]]))

            first_second_state = tensor(thermal_state(beta_one), thermal_state(beta_two))

            ground_state = Qobj([[0,0],[0,1]])


            corr_state = first_second_state + chi*corr
            var_alpha_system = tensor(corr_state, ground_state, ground_state, ground_state)
            time_evo = qutip.mesolve(H_perf, var_alpha_system, t, [], []).states
            var_alpha_evo.append(time_evo)
    else:
        print('give string or float with 0<corr<1')


    ind = 0
    print("time evo done")
    print()

    filename = "diff_beta/which_beta/"+filename

    print(filename)
    print('dot{I}^2_max \t t^star \t V_max \t beta2')
    with open("txtfiles/i_dot_sq_max/"+filename+".txt", 'a') as f:
        f.write('dot{I}^2_max \t t^star \t V_max \t beta2\n')
        for alpha_evo in var_alpha_evo:
            kld_alpha = [entropy_relative(thermal_state(beta_one),alpha_evo[dt].ptrace(N)) for dt in range(len(t))]
            vn_alpha = [entropy_vn(alpha_evo[dt].ptrace(N), base=np.e) for dt in range(len(t))]

            i_dot_sq_alpha = np.square(np.diff(vn_alpha)/dt)
            bound = [-1.j *(np.pi/(3*hbar)) * (commutator(H_perf, alpha_evo[dt]).ptrace(N) * (sigmaz())).tr() for dt in range(1,len(t))]
            bound_diff = [np.real(bound[dt]) - i_dot_sq_alpha[dt] for dt in range(len(t)-1)]

            bound_diff_min = min(bound_diff[:3138])
            i_dot_sq_alpha_max = max(i_dot_sq_alpha[:3142])
            kld_min_alpha = min(kld_alpha[:3138])
            kld_min_alpha_index=kld_alpha.index(kld_min_alpha)
            f.write(str(i_dot_sq_alpha_max)+'\t'+str(kld_min_alpha_index)+'\t'+str(bound_diff_min)+'\t'+str(.1 + 0.1*ind)+'\n')
            print(str(i_dot_sq_alpha_max)+'\t'+str(kld_min_alpha_index)+'\t'+str(bound_diff_min)+'\t'+str(.1 + .1*ind))
            ind += 1


In [ ]:
## \dot{I}^2 max value as a function of alpha and position of max fidelity
def three_maxima(beta_one, beta_two, filename):
    alpha = 1.j / (4*np.cosh(beta_one)*np.cosh(beta_two))
    chi = alpha * tensor(Qobj([[0,1],[0,0]]), Qobj([[0,0],[1,0]])) - alpha * tensor(Qobj([[0,0],[1,0]]), Qobj([[0,1],[0,0]]))
    first_second_state = tensor(thermal_state(beta_one), thermal_state(beta_two))

    var_alpha_evo = []
    ratio = np.append(np.logspace(-3,-1.1,15, endpoint=True), np.linspace(.1, 1, 30, endpoint=True))
    ratio = np.insert(ratio, 0,0)

    ground_state = Qobj([[0,0],[0,1]])

    for i in ratio:
        corr_state = first_second_state + chi*i
        var_alpha_system = tensor(corr_state, ground_state, ground_state, ground_state)
        time_evo = qutip.mesolve(H_perf, var_alpha_system, t, [], []).states
        var_alpha_evo.append(time_evo)

    ind = 0
    print("time evo done")
    print()

    filename = "verkackt/maxima_with_diff_beta"+filename

    print(filename)
    print('dot{I}^2_max \t t^star \t V_max \t alpha/alpha_max \t beta1 \t beta2')
    with open("txtfiles/i_dot_sq_max/"+filename+".txt", 'a') as f:
        f.write('dot{I}^2_max \t t^star \t V_max \t alpha/alpha_max \t beta1 \t beta2\n')
        for alpha_evo in var_alpha_evo:
            #kld_alpha = [entropy_relative(thermal_state(beta_one),alpha_evo[dt].ptrace(N)) for dt in range(len(t))]
            #vn_alpha = [entropy_vn(alpha_evo[dt].ptrace(N), base=np.e) for dt in range(len(t))]

            #i_dot_sq_alpha = np.square(np.diff(vn_alpha)/dt)
            bound = [-1.j *(np.pi/(3*hbar)) * (commutator(H_perf, alpha_evo[dt]).ptrace(N) * (sigmaz())).tr() for dt in range(1,len(t))]
            #bound_diff = [np.real(bound[dt]) - i_dot_sq_alpha[dt] for dt in range(len(t)-1)]

            #bound_diff_min = min(bound_diff[:3142])
            #i_dot_sq_alpha_max = max(i_dot_sq_alpha[:3142])
            #kld_min_alpha = min(kld_alpha[:3138])
            #kld_min_alpha_index=kld_alpha.index(kld_min_alpha)
            t_bound = np.isclose(bound[:1000], 0, atol=0.005)
            f.write(str(t_bound) +'\t'+str(ratio[ind])+'\t'+str(beta_one)+'\t'+str(beta_two)+'\n')
            print(str(t_bound)+'\t'+str(ratio[ind])+'\t'+str(beta_one)+'\t'+str(beta_two))
            ind += 1